# Blue Brain BioExplorer - Neuronmodulation covers
![](../bioexplorer_neuromodulation_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

scale = Vector3(1000, 1000, 1000)
varicosity = Vector3(59.711, 1085.713, 334.307)
output_folder = '/tmp'

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'o1'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

### Load neurons

In [ ]:
presynaptic_neuron_id = 47211
neuron_population_name = 'o1'

presynaptic_assembly_name = 'PreSynaptic Neuron'
be.remove_assembly(presynaptic_assembly_name)
neurons_assembly = be.add_assembly(presynaptic_assembly_name)
status = be.add_neurons(
    assembly_name=presynaptic_assembly_name,
    population_name=neuron_population_name,
    morphology_representation=be.morphology_representation.SEGMENT,
    morphology_color_scheme=be.morphology_color_scheme.SECTION_TYPE,
    realism_level=be.morphology_realism_level.ALL,
    synapses_type=be.neuron_synapse_type.AFFERENT, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=True, generate_externals=False,
    sql_node_filter='guid=%d' % presynaptic_neuron_id,
    scale=scale
)

In [ ]:
postsynaptic_neuron_id = 49
postsynaptic_assembly_name = 'PostSynaptic Neuron'
be.remove_assembly(postsynaptic_assembly_name)
neurons_assembly = be.add_assembly(postsynaptic_assembly_name)

with Session(engine) as session:
    data = session.execute('SELECT x,y,z FROM %s.node WHERE guid=%d' % (db_schema, postsynaptic_neuron_id))
    soma_position = data.all()[0]

status = be.add_neurons(
    assembly_name=postsynaptic_assembly_name,
    population_name=neuron_population_name,
    morphology_representation=be.morphology_representation.SEGMENT,
    morphology_color_scheme=be.morphology_color_scheme.SECTION_TYPE,
    realism_level=be.morphology_realism_level.ALL,
    synapses_type=be.neuron_synapse_type.AFFERENT, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=True, generate_externals=False,
    sql_node_filter='guid=%d' % postsynaptic_neuron_id,
    scale=scale
)

model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
tf = {
    'rotation': [0.0, 0.0, 0.0, 1.0], 
    'rotation_center': [0.0, 0.0, 0.0], 
    'scale': [1.0, 1.0, 1.0], 
    'translation': [
        scale.x * (varicosity.x - soma_position[0] - 0),
        scale.y * (varicosity.y - soma_position[1] - 0),
        scale.z * (varicosity.z - soma_position[2] - 7),
    ]
}
core.update_model(id=model_id, transformation=tf)

In [ ]:
vasculature_assembly_name = 'Vasculature'
vasculature_population_name = 'vasculature'
be.remove_assembly(vasculature_assembly_name)
vasculature_assembly = be.add_assembly(vasculature_assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=vasculature_assembly_name,
    population_name=vasculature_population_name,
    realism_level=be.morphology_realism_level.ALL,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200' % (varicosity.x, varicosity.y, varicosity.z),
    scale=scale
)

In [ ]:
astrocytes_assembly_name = 'Astrocytes'
astrocytes_population_name = 'astrocytes'
connectome_population_name = 'connectome'
be.remove_assembly(astrocytes_assembly_name)
astrocytes_assembly = be.add_assembly(astrocytes_assembly_name)
astrocytes_model = be.add_astrocytes(
    assembly_name=astrocytes_assembly_name,
    population_name=astrocytes_population_name,
    vasculature_population_name=vasculature_population_name,
    connectome_population_name=connectome_population_name,
    radius_multiplier=0.5,
    realism_level=be.morphology_realism_level.ALL,
    generate_internals=True, load_somas=True, load_dendrites=True,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200 AND guid%%3=0' % (varicosity.x, varicosity.y, varicosity.z),
    scale=scale)


In [ ]:
from bioexplorer import Protein, Volume, MolecularSystemAnimationParams

resource_folder = os.path.abspath('../../tests/test_files')
pdb_folder = os.path.join(resource_folder, 'pdb')
acetylcholin_path = os.path.join(pdb_folder, 'neuromodulation', 'acetylcholin.pdb')

def add_acetylcholin(radius, nb_molecules, radius_multiplier, frame=0):
    protein_representation = be.protein_representation.ATOMS

    acetylcholin_assembly_name = 'Acetylcholin'
    acetylcholin_name = 'Acetylcholin'
    
    acetylcholin_position = Vector3(
        scale.x * varicosity.x,
        scale.y * varicosity.y,
        scale.z * varicosity.z)

    acetylcholin = Protein(
        name=acetylcholin_name, 
        source=acetylcholin_path,
        load_non_polymer_chemicals=True, load_bonds=True, load_hydrogen=True,
        occurrences=nb_molecules,
        animation_params=MolecularSystemAnimationParams(3, (frame + 1) * 2, 0.5, (frame + 2) * 2, 1.0)
    )
    volume = Volume(
        name=acetylcholin_assembly_name,
        shape=be.assembly_shape.FILLED_SPHERE,
        shape_params=Vector3(radius, 0.0, 0.0),
        protein=acetylcholin)
    be.add_volume(
        volume=volume, 
        representation=protein_representation,
        position=acetylcholin_position,
        atom_radius_multiplier=radius_multiplier)


In [ ]:
add_acetylcholin(scale.x * 2.0, 500000, 30.0)

### Materials

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size, shading_mode, glossiness=1.0, emission=0.0, user_param=1.0):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        if mid == [be.neuron_material.AFFERENT_SYNAPSE, be.neuron_material.EFFERENT_SYNAPSE]:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.neuron_material.MITOCHONDRION:
            colors.append([0.5, 0.1, 0.6])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(6.0)
        elif mid == be.neuron_material.NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(30.0)
        elif mid == be.neuron_material.SOMA:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.neuron_material.MYELIN_SHEATH:
            colors.append([0.4, 0.3, 0.5])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(50.0)
        else:
            # Membrane
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
            
        refraction_indices.append(1.0)
        emissions.append(emission)
        user_params.append(user_param)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
def set_materials():
    palettes = ['GnBu_r', 'PuRd', 'Set1', 'Wistia', 'Greys']
    model_ids = be.get_model_ids()['ids']
    i = 0
    for model_id in model_ids:
        emission = 0.0
        if i==4:
            emission = 1.0
        set_morphology_materials(
            model_id, palettes[i], be.NB_MATERIALS_PER_MORPHOLOGY,
            be.shading_mode.PERLIN, 0.1, emission, 0.1 / scale.x)
        i += 1
    core.set_renderer()

set_materials()

In [ ]:
status = core.set_renderer(
    background_color=[0.18, 0.43, 0.41],
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.fog_start = 1.0 * scale.x
params.fog_thickness = 300.0 * scale.x
params.gi_samples = 1
params.gi_strength = 0.2
params.gi_ray_length = 1.0 * scale.x
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
params.epsilon_multiplier = 100.0
params.max_ray_depth = 3
params.show_background = False
params.use_hardware_randomizer=True
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
core.set_camera(
    orientation=[-0.070, 0.437, 0.220, 0.868],
    position=[scale.x * 77.98042386871596, scale.y * 1091.834739623779, scale.z * 348.5927433943219],
    target=[scale.x * 13.78747115930694, scale.y * 1064.041932405726, scale.z * 295.22463810933783]
)

## Snapshots

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
core.set_camera(
    orientation=[0.10042722106643472, 0.5480670250715927, 0.10408422725871022, 0.823834560414896],
    position=[70884.91927496702, 1086726.4432583167, 338806.9308694824],
    target=[2906.5292038353564, 1090506.7453414442, 310916.74450726254]
)
mm.create_snapshot(
    renderer='advanced',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_%dx%d_v1' % (size[0], size[1]),
    path=output_folder)

In [ ]:
add_acetylcholin(scale.x * 2.0, 10000000, 1.0)

In [ ]:
core.set_camera(
    orientation=[0.11414384254134172, 0.3981421943009997, 0.08577072175808344, 0.9061442267190032],
    position=[60426.19242247517, 1086103.9795371932, 336060.44751665404],
    target=[15423.078123496032, 1094517.896698822, 296171.4300538681]
)

mm.create_snapshot(
    renderer='advanced',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_%dx%d_v2' % (size[0], size[1]),
    path=output_folder)

In [ ]:
core.set_camera(
    orientation=[-0.08616460361504102, 0.5168722715162748, 0.2108410708440709,0.8252058887737016],
    position=[73092.67928336751, 1091013.0323442845, 339637.27257494023],
    target=[9012.827321822671, 1062754.583344163, 304264.48103967734],    
)

mm.create_snapshot(
    renderer='advanced',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_%dx%d_v3' % (size[0], size[1]),
    path=output_folder)

In [ ]:
core.set_camera(
    orientation=[0.11168463198861378, 0.4756995788056184, 0.09304005355047734, 0.8675136898845167],
    position=[93057.10494369318, 1081509.836538513, 352380.7404783915],
    target=[11963.415460410512, 1091597.755994066, 302306.8141172933]
)

mm.create_snapshot(
    renderer='advanced',
    size=[3840, 2160], samples_per_pixel=64,
    base_name='neuromodulation_%dx%d_v4' % (size[0], size[1]),
    path=output_folder)

## Movie

In [ ]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [-0.7576676115295892, -0.32993750640608055, -0.5630994870407762],
        'focalDistance': 1000000.0,
        'origin': [60587.346422678995, 1086165.1404973788, 333836.0443882473],
        'up': [-0.4397732093712237, 0.8956089880936011, 0.06696315976183911]
    }
     ,
    {
        'apertureRadius': 0.0,
        'direction': [-0.8371737098698813, -0.37502167711053114, -0.39811922987957327],
        'focalDistance': 1000000.0,
        'origin': [68429.12995521477, 1089034.4591921489, 336340.9125636818],
        'up': [-0.42932080875343503, 0.9015645158280192, 0.05352632035817306]
    }
]

mm.build_camera_path(key_frames, 100, 100)

In [ ]:
from tqdm import tqdm

draft = False
atom_radius_multiplier = 2.0
nb_molecules = 5000000

nb_frames = mm.get_nb_frames()
for frame in tqdm(range(nb_frames)):
    add_acetylcholin(scale.x * (0.88 + frame / nb_frames), nb_molecules, atom_radius_multiplier, frame)
    mm.set_current_frame(frame)
    if draft:
        core.set_renderer()
        import time
        time.sleep(0.1)
    else:
        mm.create_snapshot(
            renderer='depth',
            size=[3840, 2160], samples_per_pixel=2,
            base_name='%05d' % frame,
            path=os.path.join(output_folder, 'depth'))

        mm.create_snapshot(
            renderer='advanced',
            size=[3840, 2160], samples_per_pixel=64,
            base_name='%05d' % frame,
            path=os.path.join(output_folder, 'bio_explorer'))